In [1]:
# This produces the dataframe for WR

In [4]:
## Notes on the NFL Library ##
# the NFL python library seem to not work on Tuesday probably due to updates (not confirmed)
# unbalanced dataframe - pfr stats start at 2018; all other stats go back to 2017

In [6]:
## REQUIRED ACTIONS - Include in a README doc ## 
# modify the season start date in the 'get_current_week' function
# modify the number of weeks if the NFL adds regular season games to the schedule

In [243]:
# import the libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import glob
from IPython.display import display, HTML
from datetime import datetime
import nfl_data_py as nfl
import os
import re
import time
from random import sample, uniform, seed
import io
from rapidfuzz import fuzz, process

In [10]:
# Set Pandas options to display all columns in a single row without wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [12]:
# Function to calculate the current week of the NFL season
def get_current_week():
    current_date = datetime.now()
    season_start_date = datetime(2024, 9, 4)  # Update for the season start
    current_week = ((current_date - season_start_date).days // 7) + 1
    return current_week

# Define the current NFL year, week, and season type
current_year = datetime.now().year
current_week = get_current_week()
seasontype = 2 if current_week <= 18 else 3  # Regular season or playoffs

In [14]:
# define the years to pull
# nfl.import_weekly_data(years, columns, downcast)
def get_year_range(current_year, current_week, start_year=2017):
    if current_week <= 18:  # Regular season
        return list(range(start_year, current_year + 1))
    else:  # Playoffs
        return list(range(start_year, current_year))

# Use the function
years = get_year_range(current_year, current_week)

In [22]:
# define the base columns. 
base_columns = [
    'season', 'season_type', 'week', 'player_id', 'player_name', 
    'position', 'position_group', 'recent_team'
]

In [25]:
# Import the player IDs from nfl.import_ids() - without parameters
ids_data = nfl.import_ids()

# Drop the unnecessary columns
columns_to_drop = [
    'position', 'team', 'birthdate', 'age', 'draft_year', 
    'draft_round', 'draft_pick', 'draft_ovr', 'twitter_username', 
    'height', 'weight', 'college', 'db_season'
]
ids_data = ids_data.drop(columns=columns_to_drop, errors='ignore')

# Display the resulting dataframe for review
# print(f"Columns after dropping unnecessary ones: {ids_data.columns.tolist()}")
# display(ids_data)

In [27]:
# import the weekly data from nfl.import_weekly_data(years, columns, downcast)
weekly_data = nfl.import_weekly_data(
    years=years,
    columns=base_columns
)

# display(weekly_data)

Downcasting floats.


In [31]:
## Output: a dataframe of ALL NFL athletes info and ids since 2017

# Merge the two dataframes on 'player_id' and 'gsis_id'
# Align column names for merging
ids_data = ids_data.rename(columns={'gsis_id': 'player_id'})  
id_dataframe = pd.merge(weekly_data, ids_data, on='player_id', how='inner')

# Assign the resulting dataframe to a variable
all_players_id_data = id_dataframe

# Display the resulting ID dataframe
# display(all_players_id_data)

In [34]:
## Output: a dataframe of NFL WR info and ids since 2017
# extract WR from the dataframe
# Create a new dataframe with only wide receivers
wide_receiver_ids = all_players_id_data[all_players_id_data['position'] == 'WR']

# Display the resulting dataframe for review
print(f"Shape of merged dataframe: {wide_receiver_ids.shape}")

# Display the resulting dataframe for review
# display(wide_receiver_ids)

Shape of merged dataframe: (17405, 29)


In [37]:
## Output: a dataframe of NFL WR info, ids, and stats since 2017
# WR-specific columns (receiving-related)
wr_columns = [
    'receptions', 'targets', 'receiving_yards', 'receiving_tds',
    'receiving_fumbles', 'receiving_fumbles_lost',
    'receiving_air_yards', 'receiving_yards_after_catch',
    'receiving_first_downs', 'receiving_epa',
    'receiving_2pt_conversions', 'racr', 'target_share',
    'air_yards_share', 'wopr'
]

# Pull WR-specific columns from weekly data
wr_stats = nfl.import_weekly_data(
    years=years,
    columns=['player_id', 'season', 'week'] + wr_columns  # Include keys for merging
)

# Merge WR-specific stats with wide_receiver_ids
wr_ids_weekly_stats = pd.merge(
    wide_receiver_ids,
    wr_stats,
    on=['player_id', 'season', 'week'],  # Ensure correct alignment
    how='inner'
)

# Display the resulting dataframe for review
print(f"Shape of merged dataframe: {wr_ids_weekly_stats.shape}")

# Row integrity check
print(
    f"Row count matches: {wr_ids_weekly_stats.shape[0] == wide_receiver_ids.shape[0]}"
)

# display the df
display(wr_ids_weekly_stats)

# csv file
wr_ids_weekly_stats.to_csv('wr_ids_weekly_stats.csv', index=False)

Downcasting floats.
Shape of merged dataframe: (17405, 44)
Row count matches: True


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr
0,2017,REG,1,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,6,13,74.0,0,0.0,0.0,144.0,44.0,4.0,0.997088,0,0.513889,0.276596,0.342043,0.654324
1,2017,REG,2,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,3,6,21.0,0,0.0,0.0,29.0,17.0,2.0,-3.455533,0,0.724138,0.166667,0.069378,0.298565
2,2017,REG,3,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,13,15,149.0,1,0.0,0.0,138.0,45.0,6.0,7.632769,0,1.079710,0.312500,0.369973,0.727731
3,2017,REG,4,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,4,7,32.0,1,0.0,0.0,31.0,18.0,1.0,0.162141,0,1.032258,0.137255,0.070938,0.255539
4,2017,REG,5,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,6,10,51.0,0,0.0,0.0,44.0,29.0,5.0,2.428232,0,1.159091,0.227273,0.105516,0.414770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17400,2024,REG,2,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley,1,1,4.0,0,0.0,0.0,-1.0,5.0,0.0,-0.475780,0,0.000000,0.034483,-0.006579,0.047119
17401,2024,REG,9,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN
17402,2024,REG,10,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley,1,2,2.0,0,0.0,0.0,12.0,0.0,1.0,-2.246118,0,0.166667,0.060606,0.057692,0.131294
17403,2024,REG,11,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley,1,1,10.0,0,0.0,0.0,10.0,0.0,0.0,0.563583,0,1.000000,0.034483,0.080645,0.108176


In [39]:
# Output: imports the NFL next-generation stats from the nfl python library

# import the next generation stats (NGS) from nfl.import_ngs_data()
# note: ngs starts at week 0 (previous season totals) - not needed so drop those rows

# Pull NGS receiving data for the specified years
ngs_wr_df = nfl.import_ngs_data('receiving', years)

# Exclude rows where 'week' == 0 and filter for 'WR' position in one step
ngs_wr_df = ngs_wr_df[(ngs_wr_df['week'] != 0) & (ngs_wr_df['player_position'] == 'WR')]

# Drop unnecessary columns
ngs_wr_df = ngs_wr_df.drop(columns=['season_type', 'player_position', 'receptions', 'targets','player_jersey_number'], errors='ignore')

# Display the resulting dataframe
print(f"Shape of NGS WR DataFrame after dropping columns: {ngs_wr_df.shape}")
display(ngs_wr_df)

# csv file
ngs_wr_df.to_csv('ngs_wr_df.csv', index=False)

Shape of NGS WR DataFrame after dropping columns: (8249, 18)


,season,week,player_display_name,team_abbr,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_short_name
1725,2017,1,Ryan Grant,WAS,9.936667,2.894592,4.410000,7.154639,66.666667,61.0,0,11.232500,10.072361,1.160139,00-0031068,Ryan,Grant,R.Grant
1726,2017,1,Martavis Bryant,PIT,8.300000,4.122054,12.688333,33.327496,33.333333,14.0,0,0.155000,4.098278,-3.943278,00-0031373,Martavis,Bryant,M.Bryant
1729,2017,1,Jamison Crowder,WAS,7.655000,3.177793,10.540000,19.949707,42.857143,14.0,0,1.450000,1.631897,-0.181897,00-0031941,Jamison,Crowder,J.Crowder
1732,2017,1,Nelson Agholor,PHI,7.423750,2.462620,10.463750,20.274656,75.000000,86.0,1,5.611667,3.262470,2.349197,00-0031549,Nelson,Agholor,N.Agholor
1733,2017,1,John Brown,ARI,7.360000,2.751526,13.422222,28.208481,44.444444,32.0,0,-0.377500,0.961993,-1.339493,00-0031051,John,Brown,J.Brown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13323,2024,23,Xavier Worthy,KC,8.160000,4.959113,14.276250,44.737358,100.000000,157.0,2,6.250000,6.154624,0.095376,00-0039894,Xavier,Worthy,X.Worthy
13324,2024,23,DeAndre Hopkins,KC,7.676000,3.446231,11.974000,23.451761,40.000000,18.0,1,0.565000,0.798474,-0.233474,00-0030564,DeAndre,Hopkins,D.Hopkins
13325,2024,23,DeVonta Smith,PHI,7.470000,2.221577,14.752000,40.028219,80.000000,69.0,1,0.340000,0.600076,-0.260076,00-0036912,DeVonta,Smith,D.Smith
13327,2024,23,Marquise Brown,KC,4.943333,3.302615,6.356667,14.939872,33.333333,15.0,0,2.450000,3.533891,-1.083891,00-0035662,Marquise,Brown,M.Brown


In [41]:
# Output: a dataframe of NFL WR info, ids, weekly stats, and next-gen stats since 2017

# Joins wr_ids_weekly_stats dataframe with ngs_wr_df using the keys 'player_id' and 'player_gsis_id'
# Merge wr_ids_weekly_stats with ngs_wr_df using a left join
wr_ids_ngs_weekly_stats = pd.merge(
    wr_ids_weekly_stats,
    ngs_wr_df,
    left_on=['player_id', 'season', 'week'],  # Keys from wr_ids_weekly_stats
    right_on=['player_gsis_id', 'season', 'week'],  # Keys from ngs_wr_df
    how='left'  # Retain all rows from wr_ids_weekly_stats
)

# Display the shape of the resulting dataframe
print(f"Shape of merged dataframe: {wr_ids_ngs_weekly_stats.shape}")
print(f"Row count matches: {wr_ids_weekly_stats.shape[0] == wr_ids_ngs_weekly_stats.shape[0]}")

# Display a sample of the merged dataframe
display(wr_ids_ngs_weekly_stats)

# csv file
wr_ids_ngs_weekly_stats.to_csv('wr_ids_ngs_weekly_stats.csv', index=False)

Shape of merged dataframe: (17405, 60)
Row count matches: True


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,player_display_name,team_abbr,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_short_name
0,2017,REG,1,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,6,13,74.0,0,0.0,0.0,144.0,44.0,4.0,0.997088,0,0.513889,0.276596,0.342043,0.654324,Larry Fitzgerald,ARI,5.936667,2.293974,10.764615,32.677938,46.153846,74.0,0.0,7.375000,8.630824,-1.255824,00-0022921,Larry,Fitzgerald,L.Fitzgerald
1,2017,REG,2,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,3,6,21.0,0,0.0,0.0,29.0,17.0,2.0,-3.455533,0,0.724138,0.166667,0.069378,0.298565,Larry Fitzgerald,ARI,4.746667,2.808189,5.010000,7.075605,50.000000,21.0,0.0,5.873333,6.784866,-0.911533,00-0022921,Larry,Fitzgerald,L.Fitzgerald
2,2017,REG,3,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,13,15,149.0,1,0.0,0.0,138.0,45.0,6.0,7.632769,0,1.079710,0.312500,0.369973,0.727731,Larry Fitzgerald,ARI,6.556000,3.289943,9.675333,33.997048,86.666667,149.0,1.0,3.536154,3.383512,0.152642,00-0022921,Larry,Fitzgerald,L.Fitzgerald
3,2017,REG,4,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,4,7,32.0,1,0.0,0.0,31.0,18.0,1.0,0.162141,0,1.032258,0.137255,0.070938,0.255539,Larry Fitzgerald,ARI,8.400000,2.609960,5.661429,8.939165,57.142857,32.0,1.0,4.125000,3.548166,0.576834,00-0022921,Larry,Fitzgerald,L.Fitzgerald
4,2017,REG,5,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,6,10,51.0,0,0.0,0.0,44.0,29.0,5.0,2.428232,0,1.159091,0.227273,0.105516,0.414770,Larry Fitzgerald,ARI,6.392222,2.449503,4.217000,11.136346,60.000000,51.0,0.0,6.521667,5.652730,0.868937,00-0022921,Larry,Fitzgerald,L.Fitzgerald
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17400,2024,REG,2,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613104.0,40944.0,NaN,3162613.0,CorlMa00,malachi-corley-1,17777.0,NaN,1607.0,40944.0,0.0,NaN,1215291.0,Malachi Corley,malachi corley,1,1,4.0,0,0.0,0.0,-1.0,5.0,0.0,-0.475780,0,0.000000,0.034483,-0.006579,0.047119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17401,2024,REG,9,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-483

In [44]:
# Output: pro-football reference dataframe for receiving data from the python nfl library
# note: PFR data not available before 2018
# there is no position info so the data will pull WR, TE, and RB receiving data

# Define the range of years for PFR data (2018 to the current year)
pfr_years = list(range(2018, current_year))

# import pro-football reference data
pfr_rec_df = nfl.import_weekly_pfr('rec',pfr_years)

# Drop unnecessary columns
pfr_rec_df = pfr_rec_df.drop(
    columns=['game_id','pfr_game_id','receiving_int','rushing_broken_tackles', 
             'passing_drops', 'passing_drop_pct'], errors='ignore')

# display dataframe
print(f"Shape of PFR dataframe: {pfr_rec_df.shape}")
display(pfr_rec_df)

# csv file
pfr_rec_df.to_csv('pfr_rec_df.csv', index=False)

Shape of PFR dataframe: (31191, 11)


,season,week,game_type,team,opponent,pfr_player_name,pfr_player_id,receiving_broken_tackles,receiving_drop,receiving_drop_pct,receiving_rat
0,2018,1,REG,PHI,ATL,Nelson Agholor,AghoNe00,0.0,0.0,0.000,80.4
1,2018,1,REG,PHI,ATL,Zach Ertz,ErtzZa00,0.0,2.0,0.200,63.7
2,2018,1,REG,PHI,ATL,Darren Sproles,SproDa00,1.0,1.0,0.143,62.8
3,2018,1,REG,PHI,ATL,DeAndre Carter,CartDe02,0.0,0.0,0.000,108.3
4,2018,1,REG,PHI,ATL,Nick Foles,FoleNi00,0.0,0.0,0.000,118.7
...,...,...,...,...,...,...,...,...,...,...,...
4448,2024,22,SB,KC,PHI,JuJu Smith-Schuster,SmitJu00,0.0,0.0,0.000,100.0
4449,2024,22,SB,KC,PHI,Noah Gray,GrayNo00,0.0,0.0,0.000,79.2
4450,2024,22,SB,KC,PHI,Kareem Hunt,HuntKa00,0.0,0.0,0.000,87.5
4451,2024,22,SB,KC,PHI,Isiah Pacheco,PachIs00,0.0,0.0,0.000,56.2


In [48]:
# Output: a dataframe of NFL WR info, ids, weekly stats, next-gen stats, and pro-football reference data
# NOTE: unbalanced dataframe - pfr stats start at 2018

# merge the pfr_rec_df with the wr_ids_ngs_weekly_stats dataframe
# match with ids then filter out the unmatched rows as they are likely (TE)
# Step 1: Merge the dataframes with a LEFT JOIN
wr_ids_ngs_pfr_stats = pd.merge(
    wr_ids_ngs_weekly_stats,
    pfr_rec_df,  # Use the full PFR dataframe as position data is unavailable
    left_on=['pfr_id', 'season', 'week'],  # Keys from wr_ids_ngs_weekly_stats
    right_on=['pfr_player_id', 'season', 'week'],  # Keys from pfr_rec_df
    how='left'  # Retain all rows from wr_ids_ngs_weekly_stats
)

# Display the shape of the resulting dataframe
print(f"Shape of merged dataframe: {wr_ids_ngs_pfr_stats.shape}")

# Row integrity check
print(
    f"Row count matches: {wr_ids_weekly_stats.shape[0] == wr_ids_ngs_weekly_stats.shape[0] == wr_ids_ngs_pfr_stats.shape[0]}"
)

# Display the first few rows of the merged dataframe for review
display(wr_ids_ngs_pfr_stats)

# csv file
wr_ids_ngs_pfr_stats.to_csv('wr_ids_ngs_pfr_stats.csv', index=False)

Shape of merged dataframe: (17405, 69)
Row count matches: True


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,player_display_name,team_abbr,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_short_name,game_type,team,opponent,pfr_player_name,pfr_player_id,receiving_broken_tackles,receiving_drop,receiving_drop_pct,receiving_rat
0,2017,REG,1,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,6,13,74.0,0,0.0,0.0,144.0,44.0,4.0,0.997088,0,0.513889,0.276596,0.342043,0.654324,Larry Fitzgerald,ARI,5.936667,2.293974,10.764615,32.677938,46.153846,74.0,0.0,7.375000,8.630824,-1.255824,00-0022921,Larry,Fitzgerald,L.Fitzgerald,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,REG,2,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,3,6,21.0,0,0.0,0.0,29.0,17.0,2.0,-3.455533,0,0.724138,0.166667,0.069378,0.298565,Larry Fitzgerald,ARI,4.746667,2.808189,5.010000,7.075605,50.000000,21.0,0.0,5.873333,6.784866,-0.911533,00-0022921,Larry,Fitzgerald,L.Fitzgerald,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,REG,3,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,13,15,149.0,1,0.0,0.0,138.0,45.0,6.0,7.632769,0,1.079710,0.312500,0.369973,0.727731,Larry Fitzgerald,ARI,6.556000,3.289943,9.675333,33.997048,86.666667,149.0,1.0,3.536154,3.383512,0.152642,00-0022921,Larry,Fitzgerald,L.Fitzgerald,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017,REG,4,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,4,7,32.0,1,0.0,0.0,31.0,18.0,1.0,0.162141,0,1.032258,0.137255,0.070938,0.255539,Larry Fitzgerald,ARI,8.400000,2.609960,5.661429,8.939165,57.142857,32.0,1.0,4.125000,3.548166,0.576834,00-0022921,Larry,Fitzgerald,L.Fitzgerald,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017,REG,5,00-0022921,L.Fitzgerald,WR,WR,ARI,7393,b6a61b38-5cfa-46eb-b1c5-b0255d7ebaf5,9383.0,1724.0,223.0,larryfitzgerald/2506106,5528.0,6762.0,1732.0,492934.0,FitzLa00,larry-fitzgerald-1,3730.0,1661.0,NaN,6762.0,246053.0,5571.0,NaN,Larry Fitzgerald,larry fitzgerald,6,10,51.0,0,0.0,0.0,44.0,29.0,5.0,2.428232,0,1.159091,0.227273,0.105516,0.414770,Larry Fitzgerald,ARI,6.392222,2.449503,4.217000,11.136346,60.000000,51.0,0.0,6.521667,5.652730,0.868937,00-0022921,Larry,Fitzgerald,L.Fitzgerald,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17400,2024,REG,2,00-0039920,M.Corley,WR,WR,NYJ,16636,bae59933-8b94-4837-990e-f0a4ced3cdbb,26023.0,NaN,11617.0,NaN,4613

In [50]:
# Output: an ordered dataframe of NFL WR info, ids, weekly stats, next-gen stats, and pro-footeball reference data
# Output: Ordered the df by year, week, and receiving yards
# NOTE: unbalanced dataframe - pfr stats start at 2018

# Order the dataframe by season (year), week, and receiving_yards
wr_ids_ngs_pfr_stats_sorted = wr_ids_ngs_pfr_stats.sort_values(
    by=['season', 'week', 'receiving_yards'], 
    ascending=[True, True, False]  # Ascending for season and week, descending for receiving_yards
)

# Display the shape of the resulting dataframe
print(f"Shape of merged dataframe: {wr_ids_ngs_pfr_stats_sorted.shape}")

# Row integrity check
print(
    f"Row count matches: {wr_ids_weekly_stats.shape[0] == wr_ids_ngs_weekly_stats.shape[0] == wr_ids_ngs_pfr_stats.shape[0] == wr_ids_ngs_pfr_stats_sorted.shape[0]}"
)

# Display the sorted dataframe
print("Dataframe sorted by season, week, and receiving_yards:")
display(wr_ids_ngs_pfr_stats_sorted)


# Save the sorted dataframe to a csv
wr_ids_ngs_pfr_stats_sorted.to_csv('wr_ids_ngs_pfr_stats_sorted.csv', index=False)

Shape of merged dataframe: (17405, 69)
Row count matches: True
Dataframe sorted by season, week, and receiving_yards:


,season,season_type,week,player_id,player_name,position,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,player_display_name,team_abbr,avg_cushion,avg_separation,avg_intended_air_yards,percent_share_of_intended_air_yards,catch_percentage,yards,rec_touchdowns,avg_yac,avg_expected_yac,avg_yac_above_expectation,player_gsis_id,player_first_name,player_last_name,player_short_name,game_type,team,opponent,pfr_player_name,pfr_player_id,receiving_broken_tackles,receiving_drop,receiving_drop_pct,receiving_rat
214,2017,REG,1,00-0027793,A.Brown,WR,WR,PIT,9988,16e33176-b73e-49b7-b0aa-c405b47a706e,9808.0,5718.0,536.0,antoniobrown/2508061,13934.0,24171.0,NaN,1272852.0,BrowAn04,antonio-brown-1,6454.0,5698.0,NaN,24171.0,406214.0,11056.0,406214.0,Antonio Brown,antonio brown,11,11,182.0,0,1.0,0.0,90.0,92.0,8.0,10.870283,0,2.022222,0.305556,0.400000,0.738333,Antonio Brown,PIT,4.442727,4.311392,7.329091,35.293088,100.000000,182.0,0.0,9.137273,6.636465,2.500807,00-0027793,Antonio,Brown,A.Brown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
645,2017,REG,1,00-0030035,A.Thielen,WR,WR,MIN,11938,2fa2b2da-4aa9-44b5-b27e-56876dfe2ad4,13429.0,8288.0,1689.0,NaN,16460.0,27277.0,NaN,2059362.0,ThieAd00,NaN,8986.0,9054.0,308.0,27277.0,733643.0,15534.0,733643.0,Adam Thielen,adam thielen,9,10,157.0,0,0.0,0.0,105.0,59.0,4.0,6.655833,0,1.495238,0.312500,0.439331,0.776281,Adam Thielen,MIN,3.920000,2.277252,10.673000,42.259265,90.000000,157.0,0.0,6.504444,7.789293,-1.284848,00-0030035,Adam,Thielen,A.Thielen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1673,2017,REG,1,00-0033040,T.Hill,WR,WR,KC,12801,01d8aee3-e1c4-4988-970a-8c0c2d08bd83,15802.0,10799.0,3321.0,NaN,3116406.0,29399.0,NaN,2131163.0,HillTy00,NaN,11222.0,11458.0,286.0,29399.0,823156.0,18082.0,823156.0,Tyreek Hill,tyreek hill,7,8,133.0,1,0.0,0.0,94.0,55.0,5.0,9.070634,0,1.414894,0.235294,0.361538,0.606018,Tyreek Hill,KC,7.078571,5.210156,11.551250,35.725055,87.500000,133.0,1.0,7.945714,10.144182,-2.198467,00-0033040,Tyreek,Hill,T.Hill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,2017,REG,1,00-0027891,G.Tate,WR,WR,DET,9831,c88d9352-b835-45ed-a909-1cfec09a58bc,9683.0,5585.0,642.0,goldentate/497326,13217.0,24035.0,NaN,1265470.0,TateGo00,golden-tate-1,6389.0,5583.0,NaN,24035.0,400490.0,11611.0,400490.0,Golden Tate,golden tate,10,12,107.0,0,0.0,0.0,68.0,43.0,5.0,-5.569108,0,1.573529,0.307692,0.232877,0.624552,Golden Tate,DET,5.639167,2.270264,5.055833,20.000659,83.333333,107.0,0.0,4.657000,4.412235,0.244765,00-0027891,Golden,Tate,G.Tate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,2017,REG,1,00-0026035,D.Amendola,WR,WR,NE,9308,973bfe3c-6d0d-4130-a79c-f860650b1da6,9146.0,4717.0,491.0,dannyamendola/2649,11674.0,9037.0,5595.0,516968.0,AmenDa00,NaN,5813.0,4991.0,NaN,9037.0,263758.0,9906.0,263758.0,Danny Amendola,danny amendola,6,7,100.0,0,1.0,0.0,54.0,49.0,5.0,6.142825,0,1.851852,0.194444,0.095238,0.358333,Danny Amendola,NE,2.763333,3.564135,7.581429,9.771861,85.714286,100.0,0.0,8.360000,8.952487,-0.592487,00-0026035,Daniel,Amendola,D.Amendola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15248,2024,POST,22,00-0030564,D.Hopkins,WR,WR,KC,11232,5c48ade7-4b9a-4757-9643-87a6e3839e2b,11606.0,7808.0,1426.0,deandrehopkins/2540165,15795.0,26650.0,NaN,1737078.0,HopkDe00,deandre-hopkins-1,8619.0,8404.0,NaN,26650.0,560241.0,14986.0,560241.0,DeAndre Hopkins,deandre hopki

In [53]:
## Data Integrity Checks ##
# load the datframe from the variables in memory or csv files in the current working directory
def load_dataframe(var_name, file_name):
    try:
        return globals()[var_name]  # Try to get the variable from the global namespace
    except KeyError:
        print(f"⚠️ {var_name} not found in memory. Reading from {file_name}")
        return pd.read_csv(file_name)

In [56]:
## Data Integrity Checks ##
# Output: A Stratified Random Sample of years and players from the python nfl library
# test to ensure merged df values match the ids and weekly stats from the unmerged df
def test_wr_weekly_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1):
    """
    Perform integrity checks on WR Weekly Stats data by comparing merged and unmerged dataframes.
    This function checks consistency across selected numerical columns.

    Parameters:
    years (range): Range of years to test.
    num_samples (int): Number of random samples per year.
    tolerance (float): Allowed numerical difference for matching values.

    Returns:
    None
    """
    print("\n🔹 Running Weekly Stats Integrity Check...")

    # Load DataFrames
    merged_df = load_dataframe("wr_ids_ngs_pfr_stats_sorted", "wr_ids_ngs_pfr_stats_sorted.csv")
    weekly_df = load_dataframe("wr_ids_weekly_stats", "wr_ids_weekly_stats.csv")

    print(f"✅ Using {'in-memory variable' if 'wr_ids_ngs_pfr_stats_sorted' in globals() else 'CSV file'}: wr_ids_ngs_pfr_stats_sorted")
    print(f"✅ Using {'in-memory variable' if 'wr_ids_weekly_stats' in globals() else 'CSV file'}: wr_ids_weekly_stats")

    mismatches = []

    for year in years:
        print(f"\nTesting Year: {year}")

        # Filter data for the specified year
        merged_year_df = merged_df[merged_df["season"] == year]
        weekly_year_df = weekly_df[weekly_df["season"] == year]

        # Ensure numeric columns
        numeric_cols = ["receptions", "targets", "receiving_yards", "receiving_air_yards", "receiving_yards_after_catch", "receiving_first_downs"]
        merged_year_df.loc[:, numeric_cols] = merged_year_df[numeric_cols].apply(pd.to_numeric, errors="coerce")
        weekly_year_df.loc[:, numeric_cols] = weekly_year_df[numeric_cols].apply(pd.to_numeric, errors="coerce")

        # Select random sample of players for this year
        sampled_players = sample(list(merged_year_df["player_id"].dropna()), min(num_samples, len(merged_year_df)))

        for player_id in sampled_players:
            # Filter player data from each DataFrame
            merged_player = merged_year_df[merged_year_df["player_id"] == player_id]
            weekly_player = weekly_year_df[weekly_year_df["player_id"] == player_id]

            # Loop through each week and compare values
            for week in merged_player["week"].unique():
                merged_row = merged_player[merged_player["week"] == week]
                weekly_row = weekly_player[weekly_player["week"] == week]

                if merged_row.empty or weekly_row.empty:
                    continue  # Skip if there's no data for this player/week

                # Compare numerical columns
                for col in numeric_cols:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    weekly_val = weekly_row[col].values[0] if col in weekly_row else None

                    if merged_val is not None and weekly_val is not None:
                        if abs(float(merged_val) - float(weekly_val)) > tolerance:
                            mismatches.append((year, player_id, week, col, merged_val, weekly_val))

        print(f"Tested {len(sampled_players)} players in Year {year}.")

    # Summary of mismatches
    print(f"\nTotal mismatches found in Weekly Stats: {len(mismatches)}")
    if mismatches:
        print("\nMismatch Details (Year, Player ID, Week, Column, Merged Value, Weekly Value):")
        for m in mismatches[:10]:  # Print only first 10 mismatches
            print(m)
    else:
        print("✅ Weekly Stats Integrity Check Passed!")

# Run the test
test_wr_weekly_stats()


🔹 Running Weekly Stats Integrity Check...
✅ Using in-memory variable: wr_ids_ngs_pfr_stats_sorted
✅ Using in-memory variable: wr_ids_weekly_stats

Testing Year: 2017
Tested 25 players in Year 2017.

Testing Year: 2018
Tested 25 players in Year 2018.

Testing Year: 2019
Tested 25 players in Year 2019.

Testing Year: 2020
Tested 25 players in Year 2020.

Testing Year: 2021
Tested 25 players in Year 2021.

Testing Year: 2022
Tested 25 players in Year 2022.

Testing Year: 2023
Tested 25 players in Year 2023.

Total mismatches found in Weekly Stats: 0
✅ Weekly Stats Integrity Check Passed!


In [61]:
## Data Integrity Checks ##
# Output: A Stratified Random Sample of years and players from the python nfl library
# test to ensure merged df values match the next-gen stats within the unmerged df
def test_wr_ngs_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1):
    """
    Perform integrity checks on WR Next-Gen Stats (NGS) data.
    Compares key stats across merged and unmerged dataframes.
    """
    print("\n🔹 Running Next-Gen Stats (NGS) Integrity Check...")

    # Load dataframes
    merged_df = load_dataframe("wr_ids_ngs_pfr_stats_sorted", "wr_ids_ngs_pfr_stats_sorted.csv")
    ngs_df = load_dataframe("ngs_wr_df", "ngs_wr_df.csv")

    # Validation message
    print("✅ Using in-memory variable: wr_ids_ngs_pfr_stats_sorted" if "wr_ids_ngs_pfr_stats_sorted" in globals() else "✅ Loaded wr_ids_ngs_pfr_stats_sorted from CSV")
    print("✅ Using in-memory variable: ngs_wr_df" if "ngs_wr_df" in globals() else "✅ Loaded ngs_wr_df from CSV")

    mismatches = []

    for year in years:
        print(f"\nTesting Season: {year}")

        # Filter data for the given season
        merged_year_df = merged_df[merged_df["season"] == year]
        ngs_year_df = ngs_df[ngs_df["season"] == year]

        # Select random sample of players for this season
        sampled_players = sample(list(merged_year_df["player_id"].dropna()), min(num_samples, len(merged_year_df)))

        for player_id in sampled_players:
            # Filter player data from each DataFrame
            merged_player = merged_year_df[merged_year_df["player_id"] == player_id]
            ngs_player = ngs_year_df[ngs_year_df["player_gsis_id"] == player_id]

            for week in merged_player["week"].unique():
                merged_row = merged_player[merged_player["week"] == week]
                ngs_row = ngs_player[ngs_player["week"] == week]

                if merged_row.empty or ngs_row.empty:
                    continue  # Skip if there's no data for this player/week
                
                # Compare Next-Gen Stats columns
                for col in [
                    "avg_cushion", "avg_separation", "avg_intended_air_yards",
                    "percent_share_of_intended_air_yards", "catch_percentage",
                    "avg_yac", "avg_expected_yac", "avg_yac_above_expectation"
                ]:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    ngs_val = ngs_row[col].values[0] if col in ngs_row else None

                    if merged_val is not None and ngs_val is not None:
                        merged_val = float(merged_val)  # Ensure numeric conversion
                        ngs_val = float(ngs_val)  # Ensure numeric conversion

                        if abs(merged_val - ngs_val) > tolerance:
                            mismatches.append((year, player_id, week, col, merged_val, ngs_val))

        print(f"Tested {len(sampled_players)} players in Season {year}.")

    print(f"\nTotal mismatches found in Next-Gen Stats: {len(mismatches)}")

    if mismatches:
        print("\nMismatch Details (Season, Player ID, Week, Column, Merged Value, NGS Value):")
        for m in mismatches[:10]:  # Print only the first 10 mismatches
            print(m)
    else:
        print("✅ Next-Gen Stats Integrity Check Passed!")

# Run the test
test_wr_ngs_stats()


🔹 Running Next-Gen Stats (NGS) Integrity Check...
✅ Using in-memory variable: wr_ids_ngs_pfr_stats_sorted
✅ Using in-memory variable: ngs_wr_df

Testing Season: 2017
Tested 25 players in Season 2017.

Testing Season: 2018
Tested 25 players in Season 2018.

Testing Season: 2019
Tested 25 players in Season 2019.

Testing Season: 2020
Tested 25 players in Season 2020.

Testing Season: 2021
Tested 25 players in Season 2021.

Testing Season: 2022
Tested 25 players in Season 2022.

Testing Season: 2023
Tested 25 players in Season 2023.

Total mismatches found in Next-Gen Stats: 0
✅ Next-Gen Stats Integrity Check Passed!


In [64]:
## Data Integrity Checks ##
# Output: A Stratified Random Sample of years and players from the python nfl library
# test to ensure merged df values match the pro-football reference stats within the unmerged df
def test_wr_pfr_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1):
    """
    Perform integrity checks on WR data from Pro-Football Reference (PFR).
    Compares key stats across merged and unmerged dataframes.
    """
    print("\n🔹 Running Pro-Football Reference (PFR) Integrity Check...")

    # Load dataframes
    merged_df = load_dataframe("wr_ids_ngs_pfr_stats_sorted", "wr_ids_ngs_pfr_stats_sorted.csv")
    pfr_df = load_dataframe("pfr_rec_df", "pfr_rec_df.csv")

    # Validation message
    print("✅ Using in-memory variable: wr_ids_ngs_pfr_stats_sorted" if "wr_ids_ngs_pfr_stats_sorted" in globals() else "✅ Loaded wr_ids_ngs_pfr_stats_sorted from CSV")
    print("✅ Using in-memory variable: pfr_rec_df" if "pfr_rec_df" in globals() else "✅ Loaded pfr_rec_df from CSV")

    mismatches = []

    for year in years:
        print(f"\nTesting Season: {year}")

        # Filter data for the given season
        merged_year_df = merged_df[merged_df["season"] == year].copy()
        pfr_year_df = pfr_df[pfr_df["season"] == year].copy()

        # Ensure 'pfr_id' and 'pfr_player_id' are treated as strings
        merged_year_df["pfr_id"] = merged_year_df["pfr_id"].astype(str)
        pfr_year_df["pfr_player_id"] = pfr_year_df["pfr_player_id"].astype(str)

        # Select random sample of players for this season
        sampled_players = sample(list(merged_year_df["pfr_id"].dropna()), min(num_samples, len(merged_year_df)))

        for player_id in sampled_players:
            # Filter player data from each DataFrame
            merged_player = merged_year_df[merged_year_df["pfr_id"] == player_id]
            pfr_player = pfr_year_df[pfr_year_df["pfr_player_id"] == player_id]

            for week in merged_player["week"].unique():
                merged_row = merged_player[merged_player["week"] == week]
                pfr_row = pfr_player[pfr_player["week"] == week]

                if merged_row.empty or pfr_row.empty:
                    continue  # Skip if there's no data for this player/week

                # Compare PFR Stats columns
                for col in [
                    "receiving_broken_tackles", "receiving_drop", "receiving_drop_pct", "receiving_rat"
                ]:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    pfr_val = pfr_row[col].values[0] if col in pfr_row else None

                    if merged_val is not None and pfr_val is not None:
                        merged_val = float(merged_val)  # Ensure numeric conversion
                        pfr_val = float(pfr_val)  # Ensure numeric conversion

                        if abs(merged_val - pfr_val) > tolerance:
                            mismatches.append((year, player_id, week, col, merged_val, pfr_val))

        print(f"Tested {len(sampled_players)} players in Season {year}.")

    print(f"\nTotal mismatches found in PFR Stats: {len(mismatches)}")

    if mismatches:
        print("\nMismatch Details (Season, Player ID, Week, Column, Merged Value, PFR Value):")
        for m in mismatches[:10]:  # Print only the first 10 mismatches
            print(m)
    else:
        print("✅ Pro-Football Reference (PFR) Integrity Check Passed!")

# Run the test
test_wr_pfr_stats()


🔹 Running Pro-Football Reference (PFR) Integrity Check...
✅ Using in-memory variable: wr_ids_ngs_pfr_stats_sorted
✅ Using in-memory variable: pfr_rec_df

Testing Season: 2017
Tested 25 players in Season 2017.

Testing Season: 2018
Tested 25 players in Season 2018.

Testing Season: 2019
Tested 25 players in Season 2019.

Testing Season: 2020
Tested 25 players in Season 2020.

Testing Season: 2021
Tested 25 players in Season 2021.

Testing Season: 2022
Tested 25 players in Season 2022.

Testing Season: 2023
Tested 25 players in Season 2023.

Total mismatches found in PFR Stats: 0
✅ Pro-Football Reference (PFR) Integrity Check Passed!


In [67]:
### The next section scrapes data from fantasypros ###

In [69]:
# Generates a list of (year, week) combinations for web scraping.
# - 2017-2020: Weeks 1-17
# - 2021 and beyond: Weeks 1-18
def generate_year_week_combinations(start_year, end_year):
    year_week_combinations = []
    for year in range(start_year, end_year + 1):
        max_week = 17 if year <= 2020 else 18
        year_week_combinations.extend([(year, week) for week in range(1, max_week + 1)])
    return year_week_combinations

In [72]:
# output a dataframe of weekly WR fantasypros advanced stats week-by-week
# Define the function to scrape weekly WR advanced stats from Fantasy Pros
def wr_scrape_fantasypros_advanced_stats(start_year, end_year):
    
    # Generate year-week combinations
    year_week_combinations = generate_year_week_combinations(start_year, end_year)
    
    # Initialize an empty list to store data
    all_data = []

    for year, week in year_week_combinations:
        try:
            # Construct URL
            url = f"https://www.fantasypros.com/nfl/advanced-stats-wr.php?year={year}&week={week}&range=week&view=pergame"
            # print(f"Scraping data for Year: {year}, Week: {week} from {url}")
            
            # Send GET request
            response = requests.get(url)
            response.raise_for_status()

            # Parse HTML content
            soup = BeautifulSoup(response.content, 'html.parser')

            # Find table headers and rows
            table_headers = [header.text.strip() for header in soup.find('thead').find_all('th')]
            table_rows = soup.find('tbody').find_all('tr')

            # Parse each row
            for row in table_rows:
                row_data = [cell.text.strip() for cell in row.find_all('td')]
                if len(row_data) == len(table_headers):  # Ensure the row matches headers
                    all_data.append([year, week] + row_data)

            # Random delay to avoid server overload
            time.sleep(uniform(0.3, 0.9))

        except Exception as e:
            print(f"Error occurred while scraping Year: {year}, Week: {week}: {e}")

    # Convert data to DataFrame
    column_names = ['Year', 'Week'] + table_headers
    wr_fp_advanced_stats_df = pd.DataFrame(all_data, columns=column_names)

    # Drop the 'Rank' column if it exists
    if 'Rank' in wr_fp_advanced_stats_df.columns:
        wr_fp_advanced_stats_df = wr_fp_advanced_stats_df.drop(columns=['Rank'])
    
    return wr_fp_advanced_stats_df

# define the years
wr_fp_advanced_stats_df = wr_scrape_fantasypros_advanced_stats(2017, 2024)

# Display the resulting dataframe shape
print(f"Shape of the resulting dataframe: {wr_fp_advanced_stats_df.shape}")

# Display the resulting dataframe
display(wr_fp_advanced_stats_df)

# Save to CSV with the updated name
wr_fp_advanced_stats_df.to_csv("fantasypros_wr_advanced_stats.csv", index=False)

Shape of the resulting dataframe: (26789, 20)


,Year,Week,Player,G,REC,YDS,YBC,AIR,YAC,YACON,BRKTKL,TGT,CATCHABLE,DROP,RZ TGT,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS
0,2017,1,Stefon Diggs (HOU),1,7,93,76,0,17,6,0,8,7,0,2,5,2,1,0,0
1,2017,1,Tyreek Hill (MIA),1,7,133,78,0,55,1,0,8,7,0,0,4,1,1,1,1
2,2017,1,Kenny Golladay (FA),1,4,69,64,0,5,0,0,7,5,1,1,2,1,1,1,0
3,2017,1,Antonio Brown (FA),1,11,182,90,0,92,50,0,11,11,0,0,7,2,2,1,1
4,2017,1,Adam Thielen (CAR),1,9,157,92,0,65,17,0,10,10,0,0,4,4,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26784,2024,18,Mecole Hardman Jr. (KC),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26785,2024,18,Skyy Moore (KC),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26786,2024,18,Bub Means (NO),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26787,2024,18,Ja'Lynn Polk (NE),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [75]:
# Output: a dataframe of weekly WR fantasy points and % rostered data from FantasyPros
# Scrape the weekly WR fantasy points column and the % rostered from the FantasyPros website
def wr_scrape_fantasypros_fpts_rost(start_year, end_year):
    
    # Generate year-week combinations
    week_combinations = generate_year_week_combinations(start_year, end_year)

    # Initialize an empty list to store data
    all_data = []

    for year, week in week_combinations:
        # Build the URL
        url = f"https://www.fantasypros.com/nfl/stats/wr.php?year={year}&week={week}&range=week"
        # print(f"Scraping data for Year: {year}, Week: {week} from {url}")
        
        # Request the page
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Locate the table and extract data
        table = soup.find("table", {"id": "data"})
        if table:
            table_str = str(table)
            # Skip multi-level headers using skiprows
            df = pd.read_html(io.StringIO(table_str), header=1)[0]
            df["Year"] = year
            df["Week"] = week
            all_data.append(df)
        
        # Delay to avoid bombarding the server
        time.sleep(uniform(0.3, 0.9))  # Randomized delay

    # Combine all data into a single dataframe
    wr_fpts_perct_rost_df = pd.concat(all_data, ignore_index=True)

    # Drop unnecessary columns
    columns_to_drop = [
        "Rank", "REC", "TGT", "YDS", "Y/R", "LG", "20+", "TD", 
        "ATT", "YDS.1", "TD.1", "FL", "G", "FPTS/G"
    ]
    wr_fpts_perct_rost_df.drop(columns=columns_to_drop, errors="ignore", inplace=True)

    # Save to CSV
    wr_fpts_perct_rost_df.to_csv("fantasypros_wr_fpts_perct_rost.csv", index=False)

    # Display shape of the dataframe
    print(f"Shape of WR FPTS and % Rostered dataframe after column removal: {wr_fpts_perct_rost_df.shape}")
    return wr_fpts_perct_rost_df

# define the years
wr_fpts_perct_rost_df = wr_scrape_fantasypros_fpts_rost(2017, 2024)
display(wr_fpts_perct_rost_df.head())

Shape of WR FPTS and % Rostered dataframe after column removal: (24126, 5)


,Player,FPTS,ROST,Year,Week
0,Stefon Diggs (HOU),20.7,40.5%,2017,1
1,Tyreek Hill (MIA),19.8,88.7%,2017,1
2,Kenny Golladay (FA),18.9,4.0%,2017,1
3,Antonio Brown (FA),18.2,1.0%,2017,1
4,Adam Thielen (CAR),15.7,54.2%,2017,1


In [78]:
# output: a dataframe of weekly WR redzone stats from FantasyPros
# scrape the weekly WR redzone stats from the FantasyPros
def wr_scrape_fantasypros_redzone_stats(start_year, end_year):
    
    # Generate year-week combinations
    week_combinations = generate_year_week_combinations(start_year, end_year)

    # Initialize an empty list to store data
    all_data = []

    for year, week in week_combinations:
        url = f"https://www.fantasypros.com/nfl/red-zone-stats/wr.php?year={year}&range=week&week={week}"
        # print(f"Scraping data for Year: {year}, Week: {week} from {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Locate the table and extract data
        table = soup.find("table", {"id": "data"})
        if table:
            table_str = str(table)
            df = pd.read_html(io.StringIO(table_str))[0]  # Wrap table HTML in StringIO

            # Flatten the multi-level column headers
            df.columns = df.columns.droplevel(0)
            
            # Drop the extra header rows (if any)
            df = df[df['Player'] != 'Player']  # Filter out duplicate header rows

            # Add Year and Week columns
            df["Year"] = year
            df["Week"] = week
            all_data.append(df)
        
        # Delay to avoid bombarding the server
        time.sleep(uniform(0.3, 0.9))  # Randomized delay

    # Combine all data into a single dataframe
    wr_redzone_stats_df = pd.concat(all_data, ignore_index=True)

    # Drop unnecessary columns
    columns_to_drop = ['Rank', 'ATT', 'YDS', 'TD', 'PCT', 'FL', 'G', 'FPTS', 'FPTS/G', 'ROST %']
    wr_redzone_stats_df = wr_redzone_stats_df.drop(columns=columns_to_drop, errors='ignore')

    # Rename columns to add 'rz' prefix
    wr_redzone_stats_df = wr_redzone_stats_df.rename(
        columns={
            col: f"{col}_rz" for col in wr_redzone_stats_df.columns
            if col not in ['Player', 'Year', 'Week']
        }
    )

    # Save to CSV
    wr_redzone_stats_df.to_csv("fantasypros_wr_redzone_stats.csv", index=False)

    # Display shape of the dataframe
    print(f"Shape of WR Red Zone Stats dataframe: {wr_redzone_stats_df.shape}")
    return wr_redzone_stats_df

# define the years
wr_redzone_stats_df = wr_scrape_fantasypros_redzone_stats(2017, 2024)
display(wr_redzone_stats_df.head())

Shape of WR Red Zone Stats dataframe: (7159, 8)


,Player,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz,Year,Week
0,Stefon Diggs (HOU),3,3,100.0%,7.3,60.0%,2017,1
1,Bennie Fowler III (FA),2,2,100.0%,5.5,66.7%,2017,1
2,Seth Roberts (FA),1,1,100.0%,19.0,20.0%,2017,1
3,Cooper Kupp (LAR),1,1,100.0%,18.0,100.0%,2017,1
4,DeAndre Hopkins (KC),2,3,66.7%,5.5,75.0%,2017,1


In [81]:
# output: a dataframe of weekly WR advanced stats, fantasy and rosterd data, and redzone from fantasyPros
# Merge WR Advanced Stats with Fantasy Points and % Rostered
wr_adv_fp_rost_merged_df = pd.merge(
    wr_fp_advanced_stats_df,
    wr_fpts_perct_rost_df,
    on=['Player', 'Year', 'Week'],
    how='outer'
)

# Merge the resulting dataframe with Red Zone Stats
wr_adv_fp_rost_rz_merged_df = pd.merge(
    wr_adv_fp_rost_merged_df,
    wr_redzone_stats_df,
    on=['Player', 'Year', 'Week'],
    how='outer'
)

# Display the shape and a sample of the merged dataframe
print(f"Shape of the merged dataframe: {wr_adv_fp_rost_rz_merged_df.shape}")
# display(wr_adv_fp_rost_rz_merged_df.head())

# Save the merged dataframe to a CSV file
# wr_adv_fp_rost_rz_merged_df.to_csv('wr_fantasypros.csv', index=False)
# print("Merged dataframe saved as 'wr_fantasypros.csv'.")

Shape of the merged dataframe: (28329, 27)


In [84]:
# output: a dataframe of weekly WR advanced stats, fantasy and rosterd data, and redzone from fantasyPros
# Order the merged dataframe by Year, Week, and YDS
wr_adv_fp_rost_rz_merged_df_sorted = wr_adv_fp_rost_rz_merged_df.sort_values(
    by=['Year', 'Week', 'YDS'], 
    ascending=[True, True, False]  # Ascending for Year and Week, Descending for YDS
)

# Display the shape and a sample of the sorted dataframe
print(f"Shape of the sorted dataframe: {wr_adv_fp_rost_rz_merged_df_sorted.shape}")
display(wr_adv_fp_rost_rz_merged_df_sorted.head())

# Save the sorted dataframe to a CSV file
wr_adv_fp_rost_rz_merged_df_sorted.to_csv('wr_fantasypros_sorted.csv', index=False)
print("Sorted dataframe saved as 'wr_fantasypros_sorted.csv'.")

Shape of the sorted dataframe: (28329, 27)


,Year,Week,Player,G,REC,YDS,YBC,AIR,YAC,YACON,BRKTKL,TGT,CATCHABLE,DROP,RZ TGT,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,FPTS,ROST,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz
24332,2017,1,Stefon Diggs (HOU),1,7,93,76,0,17,6,0,8,7,0,2,5,2,1,0,0,20.7,40.5%,3.0,3.0,100.0%,7.3,60.0%
2402,2017,1,Brandin Cooks (DAL),1,3,88,71,0,17,4,0,7,4,1,0,3,1,1,1,1,8.8,14.6%,NaN,NaN,NaN,NaN,NaN
20581,2017,1,Nelson Agholor (BAL),1,6,86,51,0,35,18,0,8,6,0,1,3,1,1,1,1,14.6,1.6%,1.0,1.0,100.0%,5.0,100.0%
22198,2017,1,Randall Cobb (FA),1,9,85,19,0,66,9,0,13,9,0,0,4,1,0,0,0,8.5,1.9%,NaN,NaN,NaN,NaN,NaN
19499,2017,1,Michael Crabtree (FA),1,6,83,50,0,33,31,2,7,7,1,0,4,1,0,0,0,8.3,0.0%,NaN,NaN,NaN,NaN,NaN


Sorted dataframe saved as 'wr_fantasypros_sorted.csv'.


In [87]:
## Data Integrity Checks ##
# Extract unique years and weeks from each dataframe
years_merged = sorted(wr_adv_fp_rost_rz_merged_df_sorted['Year'].unique())
weeks_merged = sorted(wr_adv_fp_rost_rz_merged_df_sorted['Week'].unique())

years_redzone = sorted(wr_redzone_stats_df['Year'].unique())
weeks_redzone = sorted(wr_redzone_stats_df['Week'].unique())

years_fpts = sorted(wr_fpts_perct_rost_df['Year'].unique())
weeks_fpts = sorted(wr_fpts_perct_rost_df['Week'].unique())

years_adv_stats = sorted(wr_fp_advanced_stats_df['Year'].unique())
weeks_adv_stats = sorted(wr_fp_advanced_stats_df['Week'].unique())

# Print the results for comparison
print(f"Years in Merged DF: {years_merged}")
print(f"Years in RedZone DF: {years_redzone}")
print(f"Years in FPTS DF: {years_fpts}")
print(f"Years in Advanced Stats DF: {years_adv_stats}\n")

print(f"Weeks in Merged DF: {weeks_merged}")
print(f"Weeks in RedZone DF: {weeks_redzone}")
print(f"Weeks in FPTS DF: {weeks_fpts}")
print(f"Weeks in Advanced Stats DF: {weeks_adv_stats}")


Years in Merged DF: [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Years in RedZone DF: [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Years in FPTS DF: [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
Years in Advanced Stats DF: [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

Weeks in Merged DF: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Weeks in RedZone DF: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Weeks in FPTS DF: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Weeks in Advanced Stats DF: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [90]:
## Data Integrity Checks ##
# Output: A Stratified Random Sample of years and players from the scraped FantasyPros data
# test to ensure merged df values match the individual unmerged df of advanced stats
def test_wr_advanced_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1, random_seed=42):
    """
    Integrity check for WR advanced stats.
    
    - Uses Stratified Random Sampling by year.
    - Tests key stats (REC, YDS, YBC, AIR, YAC, TGT, CATCHABLE).
    - Reports % of matches and mismatches found.
    - Logs whether data is loaded from variables or CSV files.
    """
    seed(random_seed)  # Set seed for reproducibility
    mismatches = []
    
    print("🔹 Running Advanced Stats Integrity Check...")
    print(f"📌 Test Details: Stratified Random Sample by Year")
    print(f"📌 Columns Tested: ['REC', 'YDS', 'YBC', 'AIR', 'YAC', 'TGT', 'CATCHABLE']\n")
    
    # Load data from memory or CSV
    merged_df = load_dataframe("wr_adv_fp_rost_rz_merged_df_sorted", "wr_fantasypros_sorted.csv")
    adv_stats_df = load_dataframe("wr_fp_advanced_stats_df", "fantasypros_wr_advanced_stats.csv")
    
    # Log the source of the data
    if isinstance(merged_df, pd.DataFrame) and isinstance(adv_stats_df, pd.DataFrame):
        print("📥 Data loaded from **variables** in memory.\n")
    else:
        print("📥 Data loaded from **CSV files**.\n")

    for year in years:
        print(f"\nTesting Year: {year}")
        
        # Filter data by year
        merged_year_df = merged_df[merged_df['Year'] == year]
        adv_year_df = adv_stats_df[adv_stats_df['Year'] == year]
        
        # Skip if no data
        if merged_year_df.empty or adv_year_df.empty:
            print(f"⚠️ Skipping Year {year} (No data available)")
            continue
        
        # Select random players
        sampled_players = sample(list(merged_year_df['Player'].dropna()), min(num_samples, len(merged_year_df)))
        
        total_comparisons = 0
        match_count = 0
        
        for player in sampled_players:
            for week in merged_year_df[merged_year_df['Player'] == player]['Week'].unique():
                
                merged_row = merged_year_df[(merged_year_df['Player'] == player) & (merged_year_df['Week'] == week)]
                adv_row = adv_year_df[(adv_year_df['Player'] == player) & (adv_year_df['Week'] == week)]
                
                if merged_row.empty or adv_row.empty:
                    continue  # Skip if missing
                
                for col in ['REC', 'YDS', 'YBC', 'AIR', 'YAC', 'TGT', 'CATCHABLE']:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    adv_val = adv_row[col].values[0] if col in adv_row else None
                    
                    if merged_val is not None and adv_val is not None:
                        total_comparisons += 1
                        if merged_val == adv_val:
                            match_count += 1
                        else:
                            mismatches.append((year, player, week, col, merged_val, adv_val))
        
        # Yearly summary
        match_percentage = (match_count / total_comparisons * 100) if total_comparisons else 0
        print(f"✅ Tested {len(sampled_players)} players in Year {year} ({total_comparisons} values compared, {match_percentage:.2f}% matched)")
    
    # Final summary
    print("\n🔍 Integrity Check Summary:")
    print(f"✔ Total Comparisons: {sum(len(m) for m in mismatches) + match_count}")
    print(f"✔ Overall Match Rate: {(match_count / (match_count + len(mismatches)) * 100) if (match_count + len(mismatches)) > 0 else 0:.2f}%")
    
    if mismatches:
        print("\n❌ Mismatch Details (Showing up to 10 cases):")
        for m in mismatches[:10]:
            print(m)
    else:
        print("✅ All values matched successfully!")

# Run the test
test_wr_advanced_stats()

🔹 Running Advanced Stats Integrity Check...
📌 Test Details: Stratified Random Sample by Year
📌 Columns Tested: ['REC', 'YDS', 'YBC', 'AIR', 'YAC', 'TGT', 'CATCHABLE']

📥 Data loaded from **variables** in memory.


Testing Year: 2017
✅ Tested 25 players in Year 2017 (2415 values compared, 100.00% matched)

Testing Year: 2018
✅ Tested 25 players in Year 2018 (2310 values compared, 100.00% matched)

Testing Year: 2019
✅ Tested 25 players in Year 2019 (2443 values compared, 100.00% matched)

Testing Year: 2020
✅ Tested 25 players in Year 2020 (2457 values compared, 100.00% matched)

Testing Year: 2021
✅ Tested 25 players in Year 2021 (2940 values compared, 100.00% matched)

Testing Year: 2022
✅ Tested 25 players in Year 2022 (2975 values compared, 100.00% matched)

Testing Year: 2023
✅ Tested 25 players in Year 2023 (3150 values compared, 100.00% matched)

🔍 Integrity Check Summary:
✔ Total Comparisons: 3150
✔ Overall Match Rate: 100.00%
✅ All values matched successfully!


In [93]:
## Data Integrity Checks ##
# Output: A Stratified Random Sample of years and players from the scraped FantasyPros data
# test to ensure merged df values match the individual unmerged df fpts and rost stats
def test_wr_fpts_rost(years=range(2017, 2024), num_samples=25, tolerance=0.1, random_seed=42):
    """
    Integrity check for WR Fantasy Points and % Rostered.
    
    - Uses Stratified Random Sampling by year.
    - Tests key stats (FPTS, ROST).
    - Reports % of matches and mismatches found.
    - Logs whether data is loaded from variables or CSV files.
    """
    seed(random_seed)  # Set seed for reproducibility
    mismatches = []
    
    print("\n🔹 Running Fantasy Points & % Rostered Integrity Check...")
    print(f"📌 Test Details: Stratified Random Sample by Year")
    print(f"📌 Columns Tested: ['FPTS', 'ROST']\n")
    
    # Load data from memory or CSV
    merged_df = load_dataframe("wr_adv_fp_rost_rz_merged_df_sorted", "wr_fantasypros_sorted.csv")
    fpts_df = load_dataframe("wr_fpts_perct_rost_df", "fantasypros_wr_fpts_perct_rost.csv")
    
    # Log the source of the data
    if isinstance(merged_df, pd.DataFrame) and isinstance(fpts_df, pd.DataFrame):
        print("📥 Data loaded from **variables** in memory.\n")
    else:
        print("📥 Data loaded from **CSV files**.\n")

    for year in years:
        print(f"\nTesting Year: {year}")
        
        # Filter data by year
        merged_year_df = merged_df[merged_df['Year'] == year]
        fpts_year_df = fpts_df[fpts_df['Year'] == year]
        
        # Skip if no data
        if merged_year_df.empty or fpts_year_df.empty:
            print(f"⚠️ Skipping Year {year} (No data available)")
            continue
        
        # Select random players
        sampled_players = sample(list(merged_year_df['Player'].dropna()), min(num_samples, len(merged_year_df)))
        
        total_comparisons = 0
        match_count = 0
        
        for player in sampled_players:
            for week in merged_year_df[merged_year_df['Player'] == player]['Week'].unique():
                
                merged_row = merged_year_df[(merged_year_df['Player'] == player) & (merged_year_df['Week'] == week)]
                fpts_row = fpts_year_df[(fpts_year_df['Player'] == player) & (fpts_year_df['Week'] == week)]
                
                if merged_row.empty or fpts_row.empty:
                    continue  # Skip if missing
                
                for col in ['FPTS', 'ROST']:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    fpts_val = fpts_row[col].values[0] if col in fpts_row else None
                    
                    if merged_val is not None and fpts_val is not None:
                        total_comparisons += 1
                        if merged_val == fpts_val:
                            match_count += 1
                        else:
                            mismatches.append((year, player, week, col, merged_val, fpts_val))
        
        # Yearly summary
        match_percentage = (match_count / total_comparisons * 100) if total_comparisons else 0
        print(f"✅ Tested {len(sampled_players)} players in Year {year} ({total_comparisons} values compared, {match_percentage:.2f}% matched)")
    
    # Final summary
    print("\n🔍 Integrity Check Summary:")
    print(f"✔ Total Comparisons: {sum(len(m) for m in mismatches) + match_count}")
    print(f"✔ Overall Match Rate: {(match_count / (match_count + len(mismatches)) * 100) if (match_count + len(mismatches)) > 0 else 0:.2f}%")
    
    if mismatches:
        print("\n❌ Mismatch Details (Showing up to 10 cases):")
        for m in mismatches[:10]:
            print(m)
    else:
        print("✅ All values matched successfully!")

# Run the test
test_wr_fpts_rost()


🔹 Running Fantasy Points & % Rostered Integrity Check...
📌 Test Details: Stratified Random Sample by Year
📌 Columns Tested: ['FPTS', 'ROST']

📥 Data loaded from **variables** in memory.


Testing Year: 2017
✅ Tested 25 players in Year 2017 (676 values compared, 100.00% matched)

Testing Year: 2018
✅ Tested 25 players in Year 2018 (638 values compared, 100.00% matched)

Testing Year: 2019
✅ Tested 25 players in Year 2019 (624 values compared, 100.00% matched)

Testing Year: 2020
✅ Tested 25 players in Year 2020 (596 values compared, 100.00% matched)

Testing Year: 2021
✅ Tested 25 players in Year 2021 (560 values compared, 100.00% matched)

Testing Year: 2022
✅ Tested 25 players in Year 2022 (566 values compared, 100.00% matched)

Testing Year: 2023
✅ Tested 25 players in Year 2023 (602 values compared, 100.00% matched)

🔍 Integrity Check Summary:
✔ Total Comparisons: 602
✔ Overall Match Rate: 100.00%
✅ All values matched successfully!


In [96]:
## Data Integrity Checks ##
# Output: A Stratified Random Sample of years and players from the scraped FantasyPros data
# test to ensure merged df values match the individual unmerged df of redzone stats
def test_wr_redzone_stats(years=range(2017, 2024), num_samples=25, tolerance=0.1, random_seed=42):
    """
    Integrity check for WR Red Zone Stats.
    
    - Uses Stratified Random Sampling by year.
    - Tests key stats: 'REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz'.
    - Reports % of matches and mismatches found.
    - Logs whether data is loaded from variables or CSV files.
    """
    seed(random_seed)  # Set seed for reproducibility
    mismatches = []
    
    print("\n🔹 Running Red Zone Stats Integrity Check...")
    print(f"📌 Test Details: Stratified Random Sample by Year")
    print(f"📌 Columns Tested: ['REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz']\n")
    
    # Load data from memory or CSV
    merged_df = load_dataframe("wr_adv_fp_rost_rz_merged_df_sorted", "wr_fantasypros_sorted.csv")
    rz_df = load_dataframe("wr_redzone_stats_df", "fantasypros_wr_redzone_stats.csv")
    
    # Log the source of the data
    if isinstance(merged_df, pd.DataFrame) and isinstance(rz_df, pd.DataFrame):
        print("📥 Data loaded from **variables** in memory.\n")
    else:
        print("📥 Data loaded from **CSV files**.\n")

    for year in years:
        print(f"\nTesting Year: {year}")
        
        # Filter data by year
        merged_year_df = merged_df[merged_df['Year'] == year]
        rz_year_df = rz_df[rz_df['Year'] == year]
        
        # Skip if no data
        if merged_year_df.empty or rz_year_df.empty:
            print(f"⚠️ Skipping Year {year} (No data available)")
            continue
        
        # Select random players
        sampled_players = sample(list(merged_year_df['Player'].dropna()), min(num_samples, len(merged_year_df)))
        
        total_comparisons = 0
        match_count = 0
        
        for player in sampled_players:
            for week in merged_year_df[merged_year_df['Player'] == player]['Week'].unique():
                
                merged_row = merged_year_df[(merged_year_df['Player'] == player) & (merged_year_df['Week'] == week)]
                rz_row = rz_year_df[(rz_year_df['Player'] == player) & (rz_year_df['Week'] == week)]
                
                if merged_row.empty or rz_row.empty:
                    continue  # Skip if missing
                
                for col in ['REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz']:
                    merged_val = merged_row[col].values[0] if col in merged_row else None
                    rz_val = rz_row[col].values[0] if col in rz_row else None
                    
                    if merged_val is not None and rz_val is not None:
                        total_comparisons += 1
                        if merged_val == rz_val:
                            match_count += 1
                        else:
                            mismatches.append((year, player, week, col, merged_val, rz_val))
        
        # Yearly summary
        match_percentage = (match_count / total_comparisons * 100) if total_comparisons else 0
        print(f"✅ Tested {len(sampled_players)} players in Year {year} ({total_comparisons} values compared, {match_percentage:.2f}% matched)")
    
    # Final summary
    print("\n🔍 Integrity Check Summary:")
    print(f"✔ Total Comparisons: {sum(len(m) for m in mismatches) + match_count}")
    print(f"✔ Overall Match Rate: {(match_count / (match_count + len(mismatches)) * 100) if (match_count + len(mismatches)) > 0 else 0:.2f}%")
    
    if mismatches:
        print("\n❌ Mismatch Details (Showing up to 10 cases):")
        for m in mismatches[:10]:
            print(m)
    else:
        print("✅ All values matched successfully!")

# Run the test
test_wr_redzone_stats()


🔹 Running Red Zone Stats Integrity Check...
📌 Test Details: Stratified Random Sample by Year
📌 Columns Tested: ['REC_rz', 'TGT_rz', 'REC PCT_rz', 'Y/R_rz', 'TGT PCT_rz']

📥 Data loaded from **variables** in memory.


Testing Year: 2017
✅ Tested 25 players in Year 2017 (505 values compared, 100.00% matched)

Testing Year: 2018
✅ Tested 25 players in Year 2018 (550 values compared, 100.00% matched)

Testing Year: 2019
✅ Tested 25 players in Year 2019 (645 values compared, 100.00% matched)

Testing Year: 2020
✅ Tested 25 players in Year 2020 (625 values compared, 100.00% matched)

Testing Year: 2021
✅ Tested 25 players in Year 2021 (510 values compared, 100.00% matched)

Testing Year: 2022
✅ Tested 25 players in Year 2022 (585 values compared, 100.00% matched)

Testing Year: 2023
✅ Tested 25 players in Year 2023 (780 values compared, 100.00% matched)

🔍 Integrity Check Summary:
✔ Total Comparisons: 780
✔ Overall Match Rate: 100.00%
✅ All values matched successfully!


In [99]:
# Load the merged dataframe
wr_adv_fp_rost_rz_merged_df_sorted = load_dataframe("wr_adv_fp_rost_rz_merged_df_sorted", "wr_fantasypros_sorted.csv")

In [102]:
# Rename 'Year' column to 'Season'
wr_adv_fp_rost_rz_merged_df_sorted.rename(columns={'Year': 'Season'}, inplace=True)
print("\u2705 Successfully renamed 'Year' to 'Season'.")
display(wr_adv_fp_rost_rz_merged_df_sorted.head())

✅ Successfully renamed 'Year' to 'Season'.


,Season,Week,Player,G,REC,YDS,YBC,AIR,YAC,YACON,BRKTKL,TGT,CATCHABLE,DROP,RZ TGT,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,FPTS,ROST,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz
24332,2017,1,Stefon Diggs (HOU),1,7,93,76,0,17,6,0,8,7,0,2,5,2,1,0,0,20.7,40.5%,3.0,3.0,100.0%,7.3,60.0%
2402,2017,1,Brandin Cooks (DAL),1,3,88,71,0,17,4,0,7,4,1,0,3,1,1,1,1,8.8,14.6%,NaN,NaN,NaN,NaN,NaN
20581,2017,1,Nelson Agholor (BAL),1,6,86,51,0,35,18,0,8,6,0,1,3,1,1,1,1,14.6,1.6%,1.0,1.0,100.0%,5.0,100.0%
22198,2017,1,Randall Cobb (FA),1,9,85,19,0,66,9,0,13,9,0,0,4,1,0,0,0,8.5,1.9%,NaN,NaN,NaN,NaN,NaN
19499,2017,1,Michael Crabtree (FA),1,6,83,50,0,33,31,2,7,7,1,0,4,1,0,0,0,8.3,0.0%,NaN,NaN,NaN,NaN,NaN


In [107]:
# split the team abbreviation from the 'player' column into a separate column 
"""
- Players with no team (Free Agents marked as FA)
- Hyphenated and multi-part names
- Ensuring consistent formatting
"""
def split_player_team(df, player_col="Player"):
    df = df.copy()

    # Extract player names and team abbreviations using regex
    player_info = df[player_col].astype(str).str.extract(r'^(?P<Player>.+?)\s*\((?P<Team>[\w\s]+)\)?$')

    # Fill missing values in the 'Team' column with 'FA' for free agents
    df["Player"] = player_info["Player"].fillna(df[player_col])  # Keep original name if extraction fails
    df["Team"] = player_info["Team"].fillna("FA")

    return df

# Apply function to the dataframe
wr_fantasypros_df_final = split_player_team(wr_adv_fp_rost_rz_merged_df_sorted)

# Display sample output
display(wr_fantasypros_df_final.head(10))


,Season,Week,Player,G,REC,YDS,YBC,AIR,YAC,YACON,BRKTKL,TGT,CATCHABLE,DROP,RZ TGT,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,FPTS,ROST,REC_rz,TGT_rz,REC PCT_rz,Y/R_rz,TGT PCT_rz,Team
24332,2017,1,Stefon Diggs,1,7,93,76,0,17,6,0,8,7,0,2,5,2,1,0,0,20.7,40.5%,3.0,3.0,100.0%,7.3,60.0%,HOU
2402,2017,1,Brandin Cooks,1,3,88,71,0,17,4,0,7,4,1,0,3,1,1,1,1,8.8,14.6%,NaN,NaN,NaN,NaN,NaN,DAL
20581,2017,1,Nelson Agholor,1,6,86,51,0,35,18,0,8,6,0,1,3,1,1,1,1,14.6,1.6%,1.0,1.0,100.0%,5.0,100.0%,BAL
22198,2017,1,Randall Cobb,1,9,85,19,0,66,9,0,13,9,0,0,4,1,0,0,0,8.5,1.9%,NaN,NaN,NaN,NaN,NaN,FA
19499,2017,1,Michael Crabtree,1,6,83,50,0,33,31,2,7,7,1,0,4,1,0,0,0,8.3,0.0%,NaN,NaN,NaN,NaN,NaN,FA
21743,2017,1,Pierre Garcon,1,6,81,38,0,43,38,1,10,6,0,0,5,2,0,0,0,8.1,0.0%,NaN,NaN,NaN,NaN,NaN,FA
4561,2017,1,Chris Hogan,1,1,8,2,0,6,1,0,5,1,0,0,0,0,0,0,0,2.5,0.0%,NaN,NaN,NaN,NaN,NaN,FA
20159,2017,1,Mike Wallace,1,1,8,6,0,2,0,0,1,1,0,0,0,0,0,0,0,0.8,0.0%,NaN,NaN,NaN,NaN,NaN,FA
26173,2017,1,Trent Taylor,1,1,8,6,0,2,1,0,1,1,0,0,0,0,0,0,0,0.8,0.0%,NaN,NaN,NaN,NaN,NaN,SF
26979,2017,1,Tyler Lockett,1,1,8,8,0,0,0,0,3,1,0,1,0,0,0,0,0,0.8,38.3%,1.0,2.0,50.0%,8.0,50.0%,SEA


In [110]:
# output: a set of duplicate columns between the wr_ids_ngs_pfr_stats_sorted and  wr_fantasypros_df_final dataframes 
# Identify Duplicate Columns

# Convert column names to lowercase before comparing to account for capitalization differences
columns_wr_ids = set(map(str.lower, wr_ids_ngs_pfr_stats_sorted.columns))
columns_fp = set(map(str.lower, wr_fantasypros_df_final.columns))

# Identify duplicate column names (case insensitive)
duplicate_columns = columns_wr_ids.intersection(columns_fp)
duplicate_columns

{'season', 'team', 'week'}

In [113]:
# Convert column names to lowercase for both dataframes
wr_ids_ngs_pfr_stats_sorted_lower = {col.lower() for col in wr_ids_ngs_pfr_stats_sorted.columns}
wr_fantasypros_df_final_lower = {col.lower() for col in wr_fantasypros_df_final.columns}

# Identify duplicate column names, ignoring case
duplicate_columns = wr_ids_ngs_pfr_stats_sorted_lower.intersection(wr_fantasypros_df_final_lower)

# Display the duplicate columns (excluding 'season' and 'week' since we need those)
duplicate_columns -= {'season', 'week'}
print(f"Duplicate columns (excluding needed ones): {duplicate_columns}")

Duplicate columns (excluding needed ones): {'team'}


In [116]:
# output: a modified version of the wr_fantasypros_df_final dataframe
# modifications: drops the 'Team' column and makes all columns lowercase for easier merging

# Remove 'Team' column with case-insensitive handling
columns_to_drop = [col for col in wr_fantasypros_df_final.columns if col.lower() == 'team']
wr_fantasypros_df_tm_modified = wr_fantasypros_df_final.drop(columns=columns_to_drop)

# Convert all column names to lowercase
wr_fantasypros_df_tm_modified.columns = wr_fantasypros_df_tm_modified.columns.str.lower()

# save to a csv file
wr_fantasypros_df_tm_modified.to_csv('wr_fantasypros_df_tm_modified.csv', index=False)

# Display results
print("✅ Successfully removed 'Team' column and created modified dataframe.")
display(wr_fantasypros_df_tm_modified.head())

✅ Successfully removed 'Team' column and created modified dataframe.


,season,week,player,g,rec,yds,ybc,air,yac,yacon,brktkl,tgt,catchable,drop,rz tgt,10+ yds,20+ yds,30+ yds,40+ yds,50+ yds,fpts,rost,rec_rz,tgt_rz,rec pct_rz,y/r_rz,tgt pct_rz
24332,2017,1,Stefon Diggs,1,7,93,76,0,17,6,0,8,7,0,2,5,2,1,0,0,20.7,40.5%,3.0,3.0,100.0%,7.3,60.0%
2402,2017,1,Brandin Cooks,1,3,88,71,0,17,4,0,7,4,1,0,3,1,1,1,1,8.8,14.6%,NaN,NaN,NaN,NaN,NaN
20581,2017,1,Nelson Agholor,1,6,86,51,0,35,18,0,8,6,0,1,3,1,1,1,1,14.6,1.6%,1.0,1.0,100.0%,5.0,100.0%
22198,2017,1,Randall Cobb,1,9,85,19,0,66,9,0,13,9,0,0,4,1,0,0,0,8.5,1.9%,NaN,NaN,NaN,NaN,NaN
19499,2017,1,Michael Crabtree,1,6,83,50,0,33,31,2,7,7,1,0,4,1,0,0,0,8.3,0.0%,NaN,NaN,NaN,NaN,NaN


In [119]:
# Load the modified FantasyPros dataframe
wr_fantasypros_df_tm_modified = load_dataframe("wr_fantasypros_df_tm_modified", "wr_fantasypros_df_tm_modified.csv")

# Load the WR stats dataframe
wr_ids_ngs_pfr_stats_sorted = load_dataframe("wr_ids_ngs_pfr_stats_sorted", "wr_ids_ngs_pfr_stats_sorted.csv")

# Confirm successful loading
print("✅ Successfully loaded both dataframes.")

✅ Successfully loaded both dataframes.


In [ ]:
### Name matching section ###

In [188]:
# test
# No output - function is used in the match_players_using_variations() function to match names
# Generate name variations for better matching
def generate_name_variations(df):
    variations = {}

    for _, row in df.iterrows():
        first = row["player_first_name"]
        last = row["player_last_name"]
        full_name = row["player_display_name"]
        merge_name = row["merge_name"]
        name_set = set()

        if pd.notna(first) and pd.notna(last):
            name_set.add(f"{first} {last}")  # Full Name
            name_set.add(f"{first.lower()} {last.lower()}")  # Lowercase Full Name
            name_set.add(f"{first.title()} {last.title()}")  # Title Case
            name_set.add(f"{first[0]}. {last}")  # First Initial + Last Name
            name_set.add(f"{first[0].upper()}. {last}")  # Uppercase Initial + Last Name
            name_set.add(f"{first.replace('-', ' ')} {last.replace('-', ' ')}")  # Handle Hyphens
            name_set.add(first.replace("'", '') + " " + last.replace("'", ''))  # Remove Apostrophes

        if pd.notna(full_name):
            name_set.add(full_name.strip())

        if pd.notna(merge_name):
            name_set.add(merge_name.strip())

        variations[row["player_id"]] = name_set

    return variations

In [ ]:
# test 1
def match_players_using_variations(wr_fantasypros_df_tm_modified, wr_ids_ngs_pfr_stats_sorted):
    """
    Matches FantasyPros player names with player IDs from wr_ids_ngs_pfr_stats_sorted
    using name variations (lowercase, initials, cleaned names).
    """
    
    # Generate name variations within the function
    name_variations = generate_name_variations(wr_ids_ngs_pfr_stats_sorted)

    # Track matches
    matched_rows = []
    unmatched_rows = []

    for _, row in wr_fantasypros_df_tm_modified.iterrows():
        fantasypros_name = row["player"].lower().strip()
        matched = False

        for player_id, name_set in name_variations.items():
            if fantasypros_name in name_set:
                matched_rows.append({**row, "player_id": player_id, "potential_match": True})
                matched = True
                break  # Stop once a match is found

        if not matched:
            unmatched_rows.append({**row, "potential_match": False})

    # Convert matched and unmatched to DataFrames
    matched_fantasypros_df = pd.DataFrame(matched_rows)
    unmatched_fantasypros_df = pd.DataFrame(unmatched_rows)

    print(f"✅ Matched Rows: {len(matched_fantasypros_df)}")
    print(f"❌ Unmatched Rows: {len(unmatched_fantasypros_df)}")

    return matched_fantasypros_df, unmatched_fantasypros_df

In [197]:
# test 1 results
# ✅ Run Matching
matched_fantasypros_df, unmatched_fantasypros_df = match_players_using_variations(
    wr_fantasypros_df_tm_modified, wr_ids_ngs_pfr_stats_sorted
)

✅ Matched Rows: 22447
❌ Unmatched Rows: 5882


In [200]:
# test 2 - enahnced name-matching techniques
def normalize_first_initials(df):
    """
    Normalize names with first initials by expanding them where possible.
    """
    updated_df = df.copy()
    updated_df["normalized_player"] = updated_df["player"].apply(lambda x: re.sub(r"\b([A-Z])\.", r"\1", x))
    return updated_df

def remove_suffixes(df):
    """
    Remove suffixes like Jr., Sr., II, III, IV from player names.
    """
    suffixes = ["Jr", "Sr", "II", "III", "IV", "V"]
    updated_df = df.copy()
    updated_df["normalized_player"] = updated_df["normalized_player"].apply(
        lambda x: re.sub(rf"\b({'|'.join(suffixes)})\b", "", x).strip()
    )
    return updated_df

def first_two_word_matching(df):
    """
    Keep only the first two words of the player name.
    """
    updated_df = df.copy()
    updated_df["normalized_player"] = updated_df["normalized_player"].apply(lambda x: " ".join(x.split()[:2]))
    return updated_df

In [221]:
# test 2 - apply the enhanced name-matching techniques
def match_players_stepwise(wr_fantasypros_df_tm_modified, wr_ids_ngs_pfr_stats_sorted):
    """
    Stepwise approach for name normalization and matching.
    """
    # Generate name variations
    name_variations = generate_name_variations(wr_ids_ngs_pfr_stats_sorted)

    # Track matches
    matched_rows = []
    unmatched_rows = wr_fantasypros_df_tm_modified.copy()

    # Step 1: First Initial Normalization
    unmatched_rows = normalize_first_initials(unmatched_rows)
    matched, unmatched_rows = attempt_matching(unmatched_rows, name_variations)
    matched_rows.extend(matched)

    print(f"✅ After First Initial Normalization: Matched Rows: {len(matched_rows)}, Unmatched: {len(unmatched_rows)}")

    # Step 2: Suffix Removal
    unmatched_rows = remove_suffixes(unmatched_rows)
    matched, unmatched_rows = attempt_matching(unmatched_rows, name_variations)
    matched_rows.extend(matched)

    print(f"✅ After Suffix Removal: Matched Rows: {len(matched_rows)}, Unmatched: {len(unmatched_rows)}")

    # Step 3: First Two-Word Matching
    unmatched_rows = first_two_word_matching(unmatched_rows)
    matched, unmatched_rows = attempt_matching(unmatched_rows, name_variations)
    matched_rows.extend(matched)

    print(f"✅ After First Two-Word Matching: Matched Rows: {len(matched_rows)}, Unmatched: {len(unmatched_rows)}")

    # Convert matched to DataFrame
    matched_fantasypros_df = pd.DataFrame(matched_rows)
    unmatched_fantasypros_df = unmatched_rows

    return matched_fantasypros_df, unmatched_fantasypros_df

In [224]:
# test2 results - run the match tests
def attempt_matching(fantasypros_df, name_variations):
    """
    Attempt to match FantasyPros players using the name variations.
    """
    matched_rows = []
    unmatched_rows = []

    for _, row in fantasypros_df.iterrows():
        fantasypros_name = row["normalized_player"].lower().strip()
        matched = False

        for player_id, name_set in name_variations.items():
            if fantasypros_name in name_set:
                matched_rows.append({**row, "player_id": player_id, "potential_match": True})
                matched = True
                break

        if not matched:
            unmatched_rows.append(row)

    return matched_rows, pd.DataFrame(unmatched_rows)

# ✅ Run Matching Stepwise
matched_fantasypros_df, unmatched_fantasypros_df = match_players_stepwise(
    wr_fantasypros_df_tm_modified, wr_ids_ngs_pfr_stats_sorted
)

✅ After First Initial Normalization: Matched Rows: 23005, Unmatched: 5324
✅ After Suffix Removal: Matched Rows: 23758, Unmatched: 4571
✅ After First Two-Word Matching: Matched Rows: 25577, Unmatched: 2752


In [367]:
# test 2 - apply fuzzy matching for the remaining unmatched rows
# requires the rapidfuzz library and a pip install 
def apply_fuzzy_matching(unmatched_df, reference_df, threshold=90):
    """
    Uses fuzzy matching to compare unmatched FantasyPros player names with wr_ids_ngs_pfr_stats_sorted.
    """
    matched_rows = []
    remaining_unmatched = []

    # Extract reference names for lookup
    reference_names = reference_df["player_display_name"].tolist()
    reference_dict = {row["player_display_name"]: row["player_id"] for _, row in reference_df.iterrows()}

    # ✅ Display remaining unmatched rows before fuzzy matching
    print(f"🔍 Remaining Unmatched Before Fuzzy Matching: {len(unmatched_df)}")

    for _, row in unmatched_df.iterrows():
        fantasypros_name = row["normalized_player"]

        # Get best match
        best_match, best_score, _ = process.extractOne(
            fantasypros_name, reference_names, scorer=fuzz.ratio
        )

        # If score meets threshold, match the player
        if best_score >= threshold:
            best_player_id = reference_dict[best_match]
            matched_rows.append({
                **row, 
                "player_id": best_player_id, 
                "matched_name": best_match, 
                "similarity_score": best_score
            })
        else:
            remaining_unmatched.append(row)

    # Convert to DataFrames
    matched_df = pd.DataFrame(matched_rows)
    remaining_unmatched_df = pd.DataFrame(remaining_unmatched)

    # ✅ Check DataFrame shapes
    print(f"✅ Fuzzy Matching Applied - Matches Found: {len(matched_df)}")
    print(f"❌ Remaining Unmatched After Fuzzy Matching: {len(remaining_unmatched_df)}")
    
    # ✅ Explicitly print DataFrame info to check if empty
    print("\n📊 DataFrame Shapes:")
    print("matched_df:", matched_df.shape)
    print("remaining_unmatched_df:", remaining_unmatched_df.shape)

    # ✅ Display a sample of matches
    if not matched_df.empty:
        print("\n📌 Sample Matched Players:")
        display(matched_df[["normalized_player", "matched_name", "similarity_score"]].head(25))
    else:
        print("\n⚠️ No matches found!")

    return matched_df, remaining_unmatched_df


In [370]:
# ✅ Run Fuzzy Matching on Remaining Unmatched Players
matched_fuzzy_df, final_unmatched_df = apply_fuzzy_matching(unmatched_fantasypros_df, wr_ids_ngs_pfr_stats_sorted)


🔍 Remaining Unmatched Before Fuzzy Matching: 2752
✅ Fuzzy Matching Applied - Matches Found: 559
❌ Remaining Unmatched After Fuzzy Matching: 2193

📊 DataFrame Shapes:
matched_df: (559, 31)
remaining_unmatched_df: (2193, 28)

📌 Sample Matched Players:


,normalized_player,matched_name,similarity_score
0,Tre'Quan Smith,Tre'Quan Smith,100.0
1,Tre'Quan Smith,Tre'Quan Smith,100.0
2,Tre'Quan Smith,Tre'Quan Smith,100.0
3,Tre'Quan Smith,Tre'Quan Smith,100.0
4,Tre'Quan Smith,Tre'Quan Smith,100.0
5,Tre'Quan Smith,Tre'Quan Smith,100.0
6,Tre'Quan Smith,Tre'Quan Smith,100.0
7,Tre'Quan Smith,Tre'Quan Smith,100.0
8,Tre'Quan Smith,Tre'Quan Smith,100.0
9,Tre'Quan Smith,Tre'Quan Smith,100.0


In [374]:
# Check if unmatched player names exist in wr_ids_ngs_pfr_stats_sorted
# ✅ Step 1: Check if remaining unmatched players exist in wr_ids_ngs_pfr_stats_sorted
unmatched_names = set(final_unmatched_df["normalized_player"].str.lower().str.strip())
reference_names = set(wr_ids_ngs_pfr_stats_sorted["player_display_name"].str.lower().str.strip())

# ✅ Players found in reference dataset
found_in_reference = unmatched_names.intersection(reference_names)
not_found_anywhere = unmatched_names.difference(reference_names)

# ✅ Summary
print(f"📌 Players Found in wr_ids_ngs_pfr_stats_sorted: {len(found_in_reference)}")
print(f"❌ Players Not Found in Any Dataset: {len(not_found_anywhere)}")

# ✅ Display a sample of names found
print("\n🔍 Sample of Found Players:")
print(list(found_in_reference)[:15])

# ✅ Display a sample of truly missing players
print("\n⚠️ Sample of Unmatched Players Not in Dataset:")
print(list(not_found_anywhere)[:15])


📌 Players Found in wr_ids_ngs_pfr_stats_sorted: 0
❌ Players Not Found in Any Dataset: 135

🔍 Sample of Found Players:
[]

⚠️ Sample of Unmatched Players Not in Dataset:
['jester weah', 'stanley berryhill', 'mose frazier', 'duron carter', 'brycen tremayne', 'terrell bynum', 'tony brown', 'reggie begelton', 'landen akers', 'ajou ajou', 'cornell powell', 'rod harper', 'malcolm perry', 'kawaan baker', 'tj luther']


In [376]:
import pandas as pd

# Function to display the summary of matching results
def display_matching_summary(matched_dfs, unmatched_final_df, wr_fantasypros_df_tm_modified):
    """
    Displays a summary of all matching results, including total matches, percentage matched, 
    and percentage matched per year.
    
    Args:
        matched_dfs (list): List of matched dataframes (direct matching, stepwise normalization, fuzzy matching).
        unmatched_final_df (pd.DataFrame): Final unmatched dataframe after all matching attempts.
        wr_fantasypros_df_tm_modified (pd.DataFrame): Original FantasyPros dataframe.
    """
    # Combine all matched results
    total_matched_df = pd.concat(matched_dfs, ignore_index=True)
    
    # Total number of FantasyPros rows
    total_fantasypros_rows = len(wr_fantasypros_df_tm_modified)
    
    # Total matched and unmatched counts
    total_matched = len(total_matched_df)
    total_unmatched = len(unmatched_final_df)
    
    # Calculate total % matched
    total_matched_percentage = (total_matched / total_fantasypros_rows) * 100

    # Calculate % matched per year
    matched_per_year = total_matched_df.groupby("season").size()
    total_per_year = wr_fantasypros_df_tm_modified.groupby("season").size()
    percent_matched_per_year = (matched_per_year / total_per_year * 100).fillna(0)

    # Display results
    print("📊 **Overall Matching Summary:**")
    print(f"✅ Total Matched: {total_matched} out of {total_fantasypros_rows} ({total_matched_percentage:.2f}%)")
    print(f"❌ Total Unmatched: {total_unmatched} out of {total_fantasypros_rows}")

    print("\n📊 **Matching Breakdown:**")
    print(f"✅ Direct Name Matching: {len(matched_dfs[0])} matches")
    print(f"✅ Stepwise Normalization: {len(matched_dfs[1])} additional matches")
    print(f"✅ Fuzzy Matching: {len(matched_dfs[2])} additional matches")
    print(f"❌ Final Unmatched: {total_unmatched}")

    print("\n📊 **Percentage Matched Per Year:**")
    print(percent_matched_per_year.to_string())

    return total_matched_df

# ✅ Run Matching Summary Display
total_matched_df = display_matching_summary(
    [matched_fantasypros_df, matched_fuzzy_df, matched_fuzzy_df],  # Combined matches
    final_unmatched_df,  # Final unmatched
    wr_fantasypros_df_tm_modified  # Original FantasyPros dataframe
)


📊 **Overall Matching Summary:**
✅ Total Matched: 26695 out of 28329 (94.23%)
❌ Total Unmatched: 2193 out of 28329

📊 **Matching Breakdown:**
✅ Direct Name Matching: 25577 matches
✅ Stepwise Normalization: 559 additional matches
✅ Fuzzy Matching: 559 additional matches
❌ Final Unmatched: 2193

📊 **Percentage Matched Per Year:**
season
2017    93.839911
2018    95.799558
2019    99.226371
2020    98.768233
2021    98.570595
2022    99.031615
2023    97.982565
2024    80.855397


In [ ]:
# Next tasks
# merge the wr_ids_ngs_pfr_stats_sorted dataframe with the wr_fantasypros_df_final dataframe
# perform data integrity check tests after the merge